# Deploy R Shiny Dashboard App

### Introduction

Now that we have built and deployed model and saved out the result, we will now:


* Store Shiny dashboard assets into the deployment space.
* Deploy Shiny assets as an app and view the dashboard.

Watson Machine Learning provides deployment spaces where the user can save, configure and deploy the shiny assets. The shiny app can be deployed in a similar way to models and functions, using the [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net) package.




All of the files associated with the dashboard are contained in a zip file which is stored in data assets. CSV files stored in the zip file were generated in ***2-model-training-and-deployment*** notebook. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

### User Inputs

The user can specify the name for the space, R-shiny app deployments.
R-shiny app deployment will be stored in the deployment space. The deployment space can be created programmatically using `ibm-watson-machine-learning client`.
If a space with specified space_name already exists, user can either use the existing space.

In [1]:
# Specify a name for the space created
space_name = 'Hospital Readmission Prediction Space'

# Specify a name for the r shiny app being deployed. 
r_shiny_deployment_name='hospital-readmission-prediction-Shiny-App'


In [2]:

from ibm_watson_machine_learning import APIClient
import os
import random
import string

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

client = APIClient(wml_credentials)

### Select Deployment Space
By default, if the space with specified name already exists, the existing space will be used to store and deploy the app.

In [3]:
space_uid=""
for space in client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with ",space_name," exists . .")
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)            
            
if space_uid=="":
    print("Deployment space",space_name, "doesn't exist")
    raise Exception("Deployment space doesn't exist")

Deployment space with  Hospital Readmission Prediction Space  exists . .


### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [4]:
rshiny_spec_uid=client.software_specifications.get_id_by_name("rstudio_r4.2")
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "hospital-readmission-prediction-shiny-assets",
    client.shiny.ConfigurationMetaNames.SOFTWARE_SPEC_UID:rshiny_spec_uid,
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/hospital-readmission-prediction-analytics-dashboard.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are 1) anyone with the link 2) Authenticated users 3) Collaborators in this deployment space

In [5]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy hospital-readmission-prediction dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1},
    client.deployments.ConfigurationMetaNames.SERVING_NAME:r_shiny_deployment_name.lower().replace('-','')[:30]+''.join(random.choice( string.digits) for _ in range(3))

}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: 'd0c2252e-4283-4403-a754-a15652d95155' started

#######################################################################################


initializing
Note: The asset is associated with one of the deprecated software specification and will be removed in future. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html
..........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1ca8ddfe-2796-4ee6-b593-2b92353d0ca7'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, select ***Deployments -> Spaces -> Hospital Readmission Prediction Space -> Deployments -> hospital-readmission-prediction-Shiny-App** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [7]:
print("{HOSTNAME}"+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny')

{HOSTNAME}/ml/v4/deployments/1ca8ddfe-2796-4ee6-b593-2b92353d0ca7/r_shiny


**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2021, 2023. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**